AInewsbot.ipynb

- Automate collecting daily AI news
- Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
- Save HTML of each URL in htmldata directory
- Extract URLs from all files, create a pandas dataframe with url, title, src
- Use ChatGPT to filter only AI-related headlines by sending a prompt and formatted table of headlines
- Use SQLite to filter headlines previously seen 
- OPENAI_API_KEY should be in the environment or in a .env file
  
Alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


1. initialize
2. fetch web pages
3. parse news story urls from web pages
4. filter headlines by relevance, not previously seen
5. perform topic analysis on headlines, and ordering by topic
6. summarize individual pages as bullet points
7. from bullet points, extract top 10 most common themes and stories of the day in order of importance
8. topic analysis of bullet points, categorize bullet points as belonging to particular themes
9. for each theme, make a summary and links. Here we want to iterate to improve summaries per specific criteria.
10. combine themes and send.

In [ ]:
# import sys
# del sys.modules['ainb_const']


In [1]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, DRIVERS)
from ainb_llm import paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, fetch_openai_summary, trunc_tokens


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [2]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [3]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-08 08:54:03,999 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-08 08:54:04,001 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-08 08:54:04,001 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-08 08:54:04,001 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-08 08:54:04,002 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-08 08:54:04,002 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-08 08:5

20

In [4]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [5]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [10]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]

# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))

2024-07-08 12:51:57,396 - AInewsbot - INFO - 17
2024-07-08 12:51:57,400 - AInewsbot - INFO - htmldata/Technology - The Washington Post (07_08_2024 08_51_30 AM).html
2024-07-08 12:51:57,400 - AInewsbot - INFO - htmldata/Technology - WSJ.com (07_08_2024 08_51_28 AM).html
2024-07-08 12:51:57,400 - AInewsbot - INFO - htmldata/AI News _ VentureBeat (07_08_2024 08_51_24 AM).html
2024-07-08 12:51:57,400 - AInewsbot - INFO - htmldata/Discover and Add New Feedly AI Feeds (07_08_2024 08_51_18 AM).html
2024-07-08 12:51:57,400 - AInewsbot - INFO - htmldata/top scoring links _ multi (07_08_2024 08_51_15 AM).html
2024-07-08 12:51:57,400 - AInewsbot - INFO - htmldata/Artificial Intelligence - The Verge (07_08_2024 08_51_13 AM).html
2024-07-08 12:51:57,401 - AInewsbot - INFO - htmldata/The Register_ Enterprise Technology News and Analysis (07_08_2024 08_51_02 AM).html
2024-07-08 12:51:57,401 - AInewsbot - INFO - htmldata/Techmeme (07_08_2024 08_50_52 AM).html
2024-07-08 12:51:57,401 - AInewsbot - INFO

# Fetch and save source pages

In [ ]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


In [ ]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log("{sourcename} -> {page}")
    

# Extract news URLs from saved pages

In [11]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-07-08 12:52:01,213 - AInewsbot - INFO - Parsing html files
2024-07-08 12:52:01,215 - AInewsbot - INFO - WaPo Tech -> htmldata/Technology - The Washington Post (07_08_2024 08_51_30 AM).html
2024-07-08 12:52:01,217 - AInewsbot - INFO - parse loop - WaPo Tech
2024-07-08 12:52:01,288 - AInewsbot - INFO - parse_file - found 159 raw links
2024-07-08 12:52:01,293 - AInewsbot - INFO - parse_file - found 19 filtered links
2024-07-08 12:52:01,294 - AInewsbot - INFO - parse loop - 19 links found
2024-07-08 12:52:01,294 - AInewsbot - INFO - WSJ Tech -> htmldata/Technology - WSJ.com (07_08_2024 08_51_28 AM).html
2024-07-08 12:52:01,294 - AInewsbot - INFO - parse loop - WSJ Tech
2024-07-08 12:52:01,335 - AInewsbot - INFO - parse_file - found 510 raw links
2024-07-08 12:52:01,341 - AInewsbot - INFO - parse_file - found 8 filtered links
2024-07-08 12:52:01,341 - AInewsbot - INFO - parse loop - 8 links found
2024-07-08 12:52:01,341 - AInewsbot - INFO - VentureBeat -> htmldata/AI News _ VentureBeat

,id,src,title,url
0,0,Ars Technica,Swarm of dusty young stars found around our ga...,https://arstechnica.com/science/2024/07/swarm-...
1,1,Ars Technica,Can’t stop your cat from scratching the furnit...,https://arstechnica.com/science/2024/07/cant-s...
2,2,Ars Technica,Egalitarian oddity found in the Neolithic,https://arstechnica.com/science/2024/07/egalit...
3,3,Ars Technica,High-altitude cave used by Tibetan Buddhists y...,https://arstechnica.com/science/2024/07/high-a...
4,4,Ars Technica,"ITER fusion reactor to see further delays, wit...",https://arstechnica.com/science/2024/07/iters-...


In [13]:
# extracts all links from history where isAI=1
# useful for training dimensionality reduction
conn = sqlite3.connect('articles.db')
c = conn.cursor()

query = "select * from news_articles where isAI=1 order by id"
ai_history_df = pd.read_sql_query(query, conn)
ai_history_df

,id,src,title,url,isAI,article_date,timestamp
0,12,Bloomberg Tech,"Xperi Holder Rubric Pushes for Cost Cuts, AI U...",https://www.bloomberg.com/news/articles/2024-0...,1,2024-01-24,None
1,43,Bloomberg Tech,Apple Dials Back Car’s Self-Driving Features a...,https://www.bloomberg.com/news/articles/2024-0...,1,2024-01-24,None
2,46,Bloomberg Tech,AIAI Startup Strikes $200 Million SPAC Deal to...,https://www.bloomberg.com/news/articles/2024-0...,1,2024-01-24,None
3,61,Bloomberg Tech,"TechnologySAP Plans 8,000-Job Restructuring to...",https://www.bloomberg.com/news/articles/2024-0...,1,2024-01-24,None
4,64,Bloomberg Tech,TechnologyNvidia Alum Snags Funding for Startu...,https://www.bloomberg.com/news/articles/2024-0...,1,2024-01-24,None
...,...,...,...,...,...,...,...
50473,124499,Reddit,Al-Generated Movie Trailer - Final Justice: Th...,https://www.reddit.com/r/ChatGPT/comments/1dxx...,1,2024-07-08,2024-07-08 09:35:00
50474,124502,Reddit,myGenerativeEiAiAlgorithm,https://www.reddit.com/r/ProgrammerHumor/comme...,1,2024-07-08,2024-07-08 09:35:00
50475,124510,Reddit,Co-author of the original Transformers paper s...,https://www.reddit.com/r/ChatGPT/comments/1dy1...,1,2024-07-08,2024-07-08 09:35:00
50476,124520,The Register,Breaking the rules is in big tech's blood – no...,https://www.theregister.com/2024/07/08/opinion...,1,2024-07-08,2024-07-08 09:35:00


In [14]:
# clean up sqlite database if you want to rerun the job from a given point
conn.execute(f"delete from news_articles where timestamp > '2024-07-08 10:15'")
# conn.execute(f"delete from news_articles where id > 220230")
# Committing the changes
conn.commit()

# Close the connection
conn.close()


# Filter URLs to new AI headlines only

In [15]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df)


2024-07-08 12:52:22,923 - AInewsbot - INFO - Existing URLs: 124046
2024-07-08 12:52:22,958 - AInewsbot - INFO - New URLs: 485


In [16]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semanting meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based

In [17]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-07-08 12:52:31,014 - AInewsbot - INFO - Paginated 10 pages


20

In [18]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-07-08 12:52:31,989 - AInewsbot - INFO - start classify
2024-07-08 12:52:31,992 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,021 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,022 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,022 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,022 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,023 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,023 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,024 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,024 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:52:32,025 - AInewsbot - INFO - sent 35 items 
2024-07-08 12:52:39,390 - AInewsbot - INFO - got dict with 50 items 
2024-07-08 12:52:39,392 - AInewsbot - INFO - got dict with 35 items 
2024-07-08 12:52:39,694 - AInewsbot - INFO - got dict with 50 items 
2024-07-08 12:52:40,105 - AInewsbot - INFO - got dict with 50 items 
2024-07-08 12:52:40,828 - AInewsbot - INFO - got dict with 50 it

485


,id,isAI
0,30,False
1,43,False
2,44,False
3,49,False
4,50,False


In [19]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,30,Bloomberg Tech,Samsung Union Steps Up Pressure Over Pay With ...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-08
1,43,Bloomberg Tech,Fight Against Misinformation Suffers Defeat on...,https://www.bloomberg.com/news/newsletters/202...,False,2024-07-08
2,44,Bloomberg Tech,"Skydance, Paramount to Merge (For Real This Time)",https://www.bloomberg.com/news/newsletters/202...,False,2024-07-08
3,49,Bloomberg Tech,Zyn Imitators Rush In as Online Sales Halt Wor...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-08
4,50,Bloomberg Tech,Tencent’s Hit Stays on Top After ‘Genshin Impa...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-08


In [20]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-08 12:52:42,905 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-08 12:52:42,906 - AInewsbot - INFO - Unmatched source rows: 0


20

In [21]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in merged_df.itertuples():
    insert_article(conn, cursor, row.src, row.title,
                   row.url, row.isAI, row.date)


In [22]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")
AIdf

2024-07-08 12:52:43,050 - AInewsbot - INFO - Found 147 AI headlines


,id,src,title,url,isAI,date
0,0,Business Insider,SpaceX rockets blew away local bird habitats a...,https://www.businessinsider.com/spacex-boca-ch...,True,2024-07-08
1,1,Feedly AI,Apple’s massive AI-powered Siri upgrade isn’t ...,https://bgr.com/tech/apples-massive-ai-powered...,True,2024-07-08
2,2,Feedly AI,Small Language Models: The Future Catalyst for...,https://blog.kore.ai/why-could-small-language-...,True,2024-07-08
3,3,Feedly AI,Two Canadian stocks set to benefit from AI-dri...,https://www.theglobeandmail.com/investing/mark...,True,2024-07-08
4,4,Feedly AI,Revolutionizing Finance: Harnessing Next-Gen A...,https://thecyberexpress.com/next-gen-ai-platfo...,True,2024-07-08
...,...,...,...,...,...,...
142,142,Reddit,Al-Generated Movie Trailer - Final Justice: Th...,https://www.reddit.com/r/ChatGPT/comments/1dxx...,True,2024-07-08
143,143,Reddit,myGenerativeEiAiAlgorithm,https://www.reddit.com/r/ProgrammerHumor/comme...,True,2024-07-08
144,144,Reddit,Co-author of the original Transformers paper s...,https://www.reddit.com/r/ChatGPT/comments/1dy1...,True,2024-07-08
145,145,The Register,Breaking the rules is in big tech's blood – no...,https://www.theregister.com/2024/07/08/opinion...,True,2024-07-08


In [23]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

def unicode_to_ascii(input_string):
    # Normalize the Unicode string to NFKD form
    normalized_string = unicodedata.normalize('NFKD', input_string)
    
    # Encode to ASCII bytes, ignoring characters that cannot be converted
    ascii_bytes = normalized_string.encode('ascii', 'ignore')
    
    # Convert bytes back to a string
    ascii_string = ascii_bytes.decode('ascii')
    
    return ascii_string

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [24]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-07-08 12:52:43,195 - AInewsbot - INFO - Found 141 unique AI headlines


20

In [26]:
# map google news headlines to redirect, kind of unnecessary
from urllib.parse import urlparse

redirect_dict = {}
for row in AIdf.itertuples():
    parsed_url = urlparse(row.url)
    netloc = parsed_url.netloc
    if netloc == 'news.google.com':
        print(netloc, end=" -> ")        
        response = requests.get(row.url, allow_redirects=False)
        # The URL to which it would have redirected
        redirect_url = response.headers.get('Location')
        redirect_dict[row.url] = redirect_url
        parsed_url2 = urlparse(redirect_url)
        netloc2 = parsed_url2.netloc
        if netloc2 == 'news.google.com':
            print(netloc2, end=" -> ")
            response = requests.get(redirect_url, allow_redirects=False)
        # The URL to which it would have redirected
            redirect_url = response.headers.get('Location')
            redirect_dict[row.url] = redirect_url
        print(redirect_url)

AIdf['url'] = AIdf['url'].apply(lambda url: redirect_dict.get(url, url))


news.google.com -> news.google.com -> https://www.cio.com/article/2150142/10-ways-to-prevent-shadow-ai-disaster.html
news.google.com -> news.google.com -> https://adage.com/article/agency-news/37-ad-execs-predict-2024-second-half-trends-ai-olympics-gen-z/2568341
news.google.com -> news.google.com -> https://investorplace.com/2024/07/3-ai-stocks-that-also-pay-dividends/
news.google.com -> news.google.com -> https://www.barrons.com/articles/3-stocks-to-benefit-from-the-ai-datacenter-power-boom-7c07b498
news.google.com -> news.google.com -> https://time.com/6994739/ai-behavior-change-health-care/
news.google.com -> news.google.com -> https://www.travelawaits.com/2993617/ai-trip-planning-how-to-use-ai-to-plan-your-next-vacation/
news.google.com -> news.google.com -> https://www.newsbytesapp.com/news/science/google-deepmind-s-new-ai-training-technique-promises-significant-gains/story
news.google.com -> news.google.com -> https://www.nextgov.com/artificial-intelligence/2024/07/ai-can-enhance

news.google.com -> news.google.com -> https://www.forbes.com/sites/nikkibaird/2024/07/08/retail-this-week-soft-landings-ai-hype-and-department-store-deaths/
news.google.com -> news.google.com -> https://www.reuters.com/technology/samsung-flags-15-fold-rise-second-quarter-profit-chip-prices-climb-2024-07-04/
news.google.com -> news.google.com -> https://www.cnbc.com/2024/07/08/southeast-asia-ipo-market-fell-significantly-in-h2-2024-deloitte.html
news.google.com -> news.google.com -> https://www.carmagazine.co.uk/car-news/motoring-issues/jenoptik-safety-cameras/
news.google.com -> news.google.com -> https://www.tomsguide.com/gaming/steams-summer-sale-is-in-full-swing-heres-9-gaming-deals-id-get-now
news.google.com -> news.google.com -> https://www.pymnts.com/artificial-intelligence-2/2024/synerise-raises-8-5-million-to-expand-ai-driven-behavioral-modeling-solutions/
news.google.com -> news.google.com -> https://9to5mac.com/2024/07/08/scambaiting-ai-systems/
news.google.com -> news.google

In [27]:
AIdf

,id,src,title,url,isAI,date
0,0,Google News,10 ways to prevent shadow AI disaster,https://www.cio.com/article/2150142/10-ways-to...,True,2024-07-08
1,1,Feedly AI,"2 stocks to turn $100 into $1,000 for end of 2024",https://finbold.com/2-stocks-to-turn-100-into-...,True,2024-07-08
2,2,Google News,37 ad execs predict 2024 second half trends: A...,https://adage.com/article/agency-news/37-ad-ex...,True,2024-07-08
3,3,Google News,3 AI Stocks That Also Pay Dividends,https://investorplace.com/2024/07/3-ai-stocks-...,True,2024-07-08
4,4,Google News,3 Stocks to Benefit From the AI Datacenter Pow...,https://www.barrons.com/articles/3-stocks-to-b...,True,2024-07-08
...,...,...,...,...,...,...
136,136,Google News,Will AI Soon Help Animals Talk To Us?,https://mindmatters.ai/2024/07/will-ai-soon-he...,True,2024-07-08
137,137,Feedly AI,Worlds first Miss AI crowned and shes a hijab...,https://nypost.com/2024/07/08/lifestyle/worlds...,True,2024-07-08
138,138,Google News,YouTube Music May Soon Let You Generate Custom...,https://english.jagran.com/technology/youtube-...,True,2024-07-08
139,139,Google News,YouTube Music is piloting 'music any way you l...,https://www.techradar.com/streaming/youtube-mu...,True,2024-07-08


# Topic analysis
Here we are trying to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

In [28]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [29]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

topic_df = pd.DataFrame(response)
print(len(topic_df))
topic_df.head()


2024-07-08 12:54:12,852 - AInewsbot - INFO - start topic extraction
2024-07-08 12:54:12,854 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:54:12,855 - AInewsbot - INFO - sent 50 items 
2024-07-08 12:54:12,856 - AInewsbot - INFO - sent 41 items 
2024-07-08 12:54:27,206 - AInewsbot - INFO - got dict with 41 items 
2024-07-08 12:54:29,370 - AInewsbot - INFO - got dict with 50 items 
2024-07-08 12:54:32,349 - AInewsbot - INFO - got dict with 50 items 
2024-07-08 12:54:32,353 - AInewsbot - INFO - Processed 141 responses.
2024-07-08 12:54:32,357 - AInewsbot - INFO - end topic extraction


141


,id,topics
0,0,"[AI, disaster prevention]"
1,1,"[stocks, investment, financial planning]"
2,2,"[advertising, trends, AI, Olympics, Gen Z]"
3,3,"[AI, stocks, dividends]"
4,4,"[AI, datacenter, technology stocks]"


In [40]:
all_topics = [item for row in topic_df.itertuples() for item in row.topics]
item_counts = Counter(all_topics)
for x in item_counts.most_common():
    print(x)
    

('AI', 76)
('investment', 8)
('ChatGPT', 8)
('China', 8)
('Google', 7)
('WhatsApp', 5)
('OpenAI', 5)
('healthcare', 4)
('technology', 4)
('innovation', 4)
('entertainment', 4)
('generative AI', 4)
('Microsoft', 4)
('Artificial Intelligence', 3)
('GitHub', 3)
('Meta AI', 3)
('business trends', 3)
('stocks', 2)
('trends', 2)
('valuation', 2)
('Siri', 2)
('salary trends', 2)
('research', 2)
('robotics', 2)
('big tech', 2)
('technology applications', 2)
('HR', 2)
('technology testing', 2)
('image analysis', 2)
('language models', 2)
('transformers', 2)
('Cohere', 2)
('Aidan Gomez', 2)
('coding', 2)
('stock market', 2)
('Galaxy AI', 2)
('Live Translate', 2)
('training tech', 2)
('Hebbia', 2)
('funding', 2)
('CEO', 2)
('vacation planning', 2)
('MIT', 2)
('researchers', 2)
('databases', 2)
('Miss AI', 2)
('Morocco', 2)
('beauty pageant', 2)
('MoBot', 2)
('plant specimens', 2)
('Nvidia', 2)
('environmental impact', 2)
('telecoms companies', 2)
('scambaiting AI systems', 2)
('Tesla', 2)
('AI mo

In [60]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
evergreen = [
    "Policy and regulation",
    "AI economic impacts",
    "Robots",
    "Autonomous vehicles",
    "AI job market",
    "LLMs",
    "Healthcare",
    "Fintech",
    "Education",
    "Entertainment",
    "Startup funding",
    "IPOs",
    "Ethical issues",
    "Legal issues",
    "Cybersecurity",
    "AI doom",
    'Stocks',
    'Climate',
    'Scams',
    'Privacy',
    'Intellectual Property',
    'Code assistants',
    'Customer service',
    'Reinforcement Learning',
    'Open Source',
    'Language Models',
    'China',
    'Military',
    'Semiconductor chips',
    'Sustainability',
    'Agriculture',
    'Gen AI',
    'Testing',
    
    'Nvidia',
    'Google',
    'OpenAI',
    'Meta',
    'Apple',
    'Microsoft',
    'Salesforce',
    'Uber',
    'AMD',
    'Netflix',
    'Disney',
    'Amazon',
    'Cloudflare',
    'Anthropic',
    'Cohere',
    'Baidu',
    'Big Tech',
    'Samsung',
    'Tesla',
    
    'ChatGPT',
    'WhatsApp',
    'Gemini',
    'Claude',
    'Copilot',
    
    'Elon Musk',
    'Bill Gates',
    'Sam Altman',
    'Mustafa Suleyman',
    'Sundar Pichai',
    'Yann LeCun',
    'Geoffrey Hinton',
    'Mark Zuckerberg',
]

In [63]:
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(evergreen)
categories


['AI doom',
 'AI economic impacts',
 'AI job market',
 'AMD',
 'Agriculture',
 'Amazon',
 'Analyst',
 'Anthropic',
 'Apple',
 'Autonomous vehicles',
 'Baidu',
 'Big Tech',
 'Bill Gates',
 'ChatGPT',
 'China',
 'Claude',
 'Climate',
 'Cloudflare',
 'Code assistants',
 'Cohere',
 'Customer service',
 'Cybersecurity',
 'Disney',
 'Education',
 'Elon Musk',
 'Entertainment',
 'Ethical issues',
 'Fintech',
 'Gemini',
 'GenAI',
 'Geoffrey Hinton',
 'Google',
 'Healthcare',
 'IPOs',
 'Intellectual Property',
 'LLMs',
 'Language Models',
 'Legal issues',
 'Mark Zuckerberg',
 'Meta',
 'Microsoft',
 'Military',
 'Mustafa Suleyman',
 'Netflix',
 'Nvidia',
 'Open Source',
 'OpenAI',
 'Policy and regulation',
 'Privacy',
 'Reinforcement Learning',
 'Robots',
 'Salesforce',
 'Sam Altman',
 'Samsung',
 'Scams',
 'Semiconductor chips',
 'Startup funding',
 'Stocks',
 'Sundar Pichai',
 'Sustainability',
 'Tesla',
 'Testing',
 'Uber',
 'WhatsApp',
 'Yann LeCun']

In [64]:
async def categorize_story(headline, categories, session, 
                           model=LOWCOST_MODEL,
                           temperature=0.5,
                           max_retries=MAX_RETRIES):
    
    retlist = []
    if type(categories) is not list:
        categories = [categories]
    for topic in categories:
        cat_prompt = f"""You are a news topic categorizaton assistant. I will provide a headline 
and a topic. You will respond with a JSON object {{'response': 1}} if the news headline matches 
the news topic and {{'response': 0}} if it does not. Check carefully and only return {{'response': 1}}
if the headline closely matches the topic. If the headline is not a close match or if unsure, 
return {{'response': 0}}
Headline:
{headline}
Topic:
{topic}
"""
        for i in range(max_retries):
            try:
                messages=[
                          {"role": "user", "content": cat_prompt
                          }]

                payload = {"model":  model,
                           "response_format": {"type": "json_object"},
                           "messages": messages,
                           "temperature": temperature
                           }
                response = await fetch_openai(session, payload)
                response_dict = json.loads(response["choices"][0]["message"]["content"])
                response_val = response_dict['response']
                if response_val == 1:
                    retlist.append(topic)
                break
            except Exception as exc:
                log(f"Error: {exc}")

            
    return retlist
        

h = "Utility stocks are Wall Streets secret backdoor to AI"
catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_story(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        


2024-07-08 13:12:33,745 - AInewsbot - INFO - Categorizing headline 1 of 141
2024-07-08 13:12:33,746 - AInewsbot - INFO - 10 ways to prevent shadow AI disaster
2024-07-08 13:12:34,945 - AInewsbot - INFO - ['AI doom', 'Ethical issues']
2024-07-08 13:12:34,947 - AInewsbot - INFO - Categorizing headline 2 of 141
2024-07-08 13:12:34,948 - AInewsbot - INFO - 2 stocks to turn $100 into $1,000 for end of 2024
2024-07-08 13:12:35,833 - AInewsbot - INFO - ['Stocks']
2024-07-08 13:12:35,835 - AInewsbot - INFO - Categorizing headline 3 of 141
2024-07-08 13:12:35,836 - AInewsbot - INFO - 37 ad execs predict 2024 second half trends: AI, Olympics, Gen Z
2024-07-08 13:12:36,649 - AInewsbot - INFO - ['GenAI']
2024-07-08 13:12:36,651 - AInewsbot - INFO - Categorizing headline 4 of 141
2024-07-08 13:12:36,651 - AInewsbot - INFO - 3 AI Stocks That Also Pay Dividends
2024-07-08 13:12:37,315 - AInewsbot - INFO - ['Stocks']
2024-07-08 13:12:37,316 - AInewsbot - INFO - Categorizing headline 5 of 141
2024-07-0

2024-07-08 13:13:02,414 - AInewsbot - INFO - CIOs' concerns over generative AI echo those of the early days of cloud computing
2024-07-08 13:13:03,182 - AInewsbot - INFO - ['GenAI']
2024-07-08 13:13:03,184 - AInewsbot - INFO - Categorizing headline 32 of 141
2024-07-08 13:13:03,185 - AInewsbot - INFO - Can AI Help With Special Ed.? There's Promiseand Reason to Be Cautious
2024-07-08 13:13:04,843 - AInewsbot - INFO - ['Education', 'Meta']
2024-07-08 13:13:04,845 - AInewsbot - INFO - Categorizing headline 33 of 141
2024-07-08 13:13:04,845 - AInewsbot - INFO - Can ChatGPT handle HR? Here's what happened when we put it to the test
2024-07-08 13:13:05,793 - AInewsbot - INFO - ['ChatGPT', 'GenAI', 'Language Models', 'OpenAI', 'Testing']
2024-07-08 13:13:05,795 - AInewsbot - INFO - Categorizing headline 34 of 141
2024-07-08 13:13:05,795 - AInewsbot - INFO - Can ChatGPT handle HR? Heres what happened when we put it to the test
2024-07-08 13:13:06,995 - AInewsbot - INFO - ['AI job market', 'Cha

2024-07-08 13:13:29,273 - AInewsbot - INFO - Categorizing headline 58 of 141
2024-07-08 13:13:29,274 - AInewsbot - INFO - Google's AI Aftershock: Expert Explains How to Thrive in the New Search Landscape
2024-07-08 13:13:29,995 - AInewsbot - INFO - ['AI economic impacts', 'Big Tech', 'Google']
2024-07-08 13:13:29,997 - AInewsbot - INFO - Categorizing headline 59 of 141
2024-07-08 13:13:29,998 - AInewsbot - INFO - Google's AI search summaries use 10x more energy than just doing a normal Google search
2024-07-08 13:13:30,696 - AInewsbot - INFO - ['AI doom', 'AI economic impacts', 'Big Tech', 'Climate', 'Ethical issues', 'Google', 'Sustainability']
2024-07-08 13:13:30,697 - AInewsbot - INFO - Categorizing headline 60 of 141
2024-07-08 13:13:30,697 - AInewsbot - INFO - Google Experts Warn That AI May Distort Reality, While AI Overviews Repel Mobile Users
2024-07-08 13:13:31,777 - AInewsbot - INFO - ['AI doom', 'AI economic impacts', 'Big Tech', 'Ethical issues', 'GenAI', 'Google', 'Languag

2024-07-08 13:13:59,197 - AInewsbot - INFO - ['AI economic impacts', 'AI job market', 'GenAI', 'Meta']
2024-07-08 13:13:59,198 - AInewsbot - INFO - Categorizing headline 85 of 141
2024-07-08 13:13:59,199 - AInewsbot - INFO - Machine learning identifies cancer-driving mutations at CTCF binding sites
2024-07-08 13:14:00,886 - AInewsbot - INFO - ['GenAI', 'Healthcare', 'Meta']
2024-07-08 13:14:00,888 - AInewsbot - INFO - Categorizing headline 86 of 141
2024-07-08 13:14:00,889 - AInewsbot - INFO - Mary Meeker urges higher ed to lead in AI
2024-07-08 13:14:02,492 - AInewsbot - INFO - ['AI economic impacts', 'AI job market', 'Education', 'GenAI']
2024-07-08 13:14:02,493 - AInewsbot - INFO - Categorizing headline 87 of 141
2024-07-08 13:14:02,494 - AInewsbot - INFO - Meet hijab-clad Kenza Layli from Morcco, world's first-ever Miss AI who beat 1500 contestants
2024-07-08 13:14:03,626 - AInewsbot - INFO - ['Entertainment', 'GenAI']
2024-07-08 13:14:03,628 - AInewsbot - INFO - Categorizing headl

2024-07-08 13:14:27,746 - AInewsbot - INFO - Telecoms companies now using scambaiting AI systems; Lenny+
2024-07-08 13:14:28,392 - AInewsbot - INFO - ['AI economic impacts', 'Cybersecurity', 'GenAI', 'Scams']
2024-07-08 13:14:28,394 - AInewsbot - INFO - Categorizing headline 113 of 141
2024-07-08 13:14:28,396 - AInewsbot - INFO - Telecoms companies now using scambaiting AI systems modelled on Lenny
2024-07-08 13:14:29,636 - AInewsbot - INFO - ['Cybersecurity', 'Language Models', 'Scams']
2024-07-08 13:14:29,637 - AInewsbot - INFO - Categorizing headline 114 of 141
2024-07-08 13:14:29,638 - AInewsbot - INFO - Tesla Bull Says EV Giant's FSD Revenue Nearing $1B Annual Run-Rate, Urges Breaking Out AI And Robotics Numbers
2024-07-08 13:14:30,503 - AInewsbot - INFO - ['AI economic impacts', 'Analyst', 'Autonomous vehicles', 'Elon Musk', 'Robots', 'Stocks', 'Tesla']
2024-07-08 13:14:30,504 - AInewsbot - INFO - Categorizing headline 115 of 141
2024-07-08 13:14:30,505 - AInewsbot - INFO - Tesla

2024-07-08 13:14:53,887 - AInewsbot - INFO - ['Big Tech', 'Entertainment', 'GenAI', 'Google', 'Language Models', 'Testing']
2024-07-08 13:14:53,889 - AInewsbot - INFO - Categorizing headline 141 of 141
2024-07-08 13:14:53,890 - AInewsbot - INFO - myGenerativeEiAiAlgorithm
2024-07-08 13:14:54,710 - AInewsbot - INFO - ['Code assistants', 'Cohere', 'GenAI', 'OpenAI']


In [65]:
catdict

{0: ['AI doom', 'Ethical issues'],
 1: ['Stocks'],
 2: ['GenAI'],
 3: ['Stocks'],
 4: ['AI economic impacts', 'Big Tech', 'Nvidia', 'Stocks'],
 5: ['GenAI', 'Healthcare'],
 6: ['GenAI', 'Military'],
 7: ['Ethical issues', 'GenAI'],
 8: ['GenAI'],
 9: ['Big Tech', 'Google', 'Reinforcement Learning', 'Sustainability'],
 10: ['AI economic impacts', 'Healthcare', 'Policy and regulation'],
 11: ['AI doom', 'Ethical issues', 'GenAI', 'OpenAI'],
 12: ['AI doom',
  'Big Tech',
  'Ethical issues',
  'Legal issues',
  'Policy and regulation',
  'Privacy'],
 13: ['AI economic impacts'],
 14: ['Ethical issues'],
 15: ['GenAI', 'Startup funding'],
 16: ['GenAI',
  'Meta',
  'OpenAI',
  'Reinforcement Learning',
  'Robots',
  'Semiconductor chips'],
 17: ['AI economic impacts', 'Cohere', 'Nvidia', 'Startup funding'],
 18: [],
 19: ['Entertainment',
  'GenAI',
  'Language Models',
  'Reinforcement Learning',
  'Robots'],
 20: ['AI economic impacts', 'GenAI'],
 21: ['Apple', 'Big Tech'],
 22: [],
 23:

In [71]:
AIdf = AIdf.drop(columns=["assigned_topics"])

In [72]:
topic_df["assigned_topics"] = topic_df["id"].apply(lambda id: catdict.get(id, []))
topic_df

,id,topics,topic_str,assigned_topics
0,0,"[AI, disaster prevention]",['disaster prevention'],"[AI doom, Ethical issues]"
1,1,"[stocks, investment, financial planning]","['stocks', 'investment', 'financial planning']",[Stocks]
2,2,"[advertising, trends, AI, Olympics, Gen Z]","['advertising', 'trends', 'Olympics', 'Gen Z']",[GenAI]
3,3,"[AI, stocks, dividends]","['stocks', 'dividends']",[Stocks]
4,4,"[AI, datacenter, technology stocks]","['datacenter', 'technology stocks']","[AI economic impacts, Big Tech, Nvidia, Stocks]"
...,...,...,...,...
136,136,"[AI, animal communication, technology impact]","['animal communication', 'technology impact']",[]
137,137,"[Miss AI, activist, Morocco, beauty pageant]","['Miss AI', 'activist', 'Morocco', 'beauty pag...",[GenAI]
138,138,"[YouTube Music, radio generation, natural lang...","['YouTube Music', 'radio generation', 'natural...","[Big Tech, GenAI, Google, Language Models]"
139,139,"[YouTube Music, music radio, AI-generated, ent...","['YouTube Music', 'music radio', 'AI-generated...","[Big Tech, Entertainment, GenAI, Google, Langu..."


In [93]:
def clean_topics(row):
    topics = [x.title() for x in row.topics if x.lower() not in {"ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    
    return ", ".join(combined)

topic_df["clean_topics"] = topic_df.apply(clean_topics, axis=1)
topic_df

,id,topics,topic_str,assigned_topics,clean_topics
0,0,"[AI, disaster prevention]","['AI Doom', 'Disaster Prevention', 'Ethical Is...","[AI doom, Ethical issues]","AI Doom, Disaster Prevention, Ethical Issues"
1,1,"[stocks, investment, financial planning]","['Financial Planning', 'Investment', 'Stocks']",[Stocks],"Financial Planning, Investment, Stocks"
2,2,"[advertising, trends, AI, Olympics, Gen Z]","['Advertising', 'Gen Z', 'Gen AI', 'Olympics',...",[GenAI],"Advertising, Gen Z, Gen AI, Olympics, Trends"
3,3,"[AI, stocks, dividends]","['Dividends', 'Stocks']",[Stocks],"Dividends, Stocks"
4,4,"[AI, datacenter, technology stocks]","['AI Economic Impacts', 'Big Tech', 'Datacente...","[AI economic impacts, Big Tech, Nvidia, Stocks]","AI Economic Impacts, Big Tech, Datacenter, Nvi..."
...,...,...,...,...,...
136,136,"[AI, animal communication, technology impact]","['Animal Communication', 'Technology Impact']",[],"Animal Communication, Technology Impact"
137,137,"[Miss AI, activist, Morocco, beauty pageant]","['Activist', 'Beauty Pageant', 'Gen AI', 'Miss...",[GenAI],"Activist, Beauty Pageant, Gen AI, Miss AI, Mor..."
138,138,"[YouTube Music, radio generation, natural lang...","['AI Technology', 'Big Tech', 'Gen AI', 'Googl...","[Big Tech, GenAI, Google, Language Models]","AI Technology, Big Tech, Gen AI, Google, Langu..."
139,139,"[YouTube Music, music radio, AI-generated, ent...","['AI-Generated', 'Big Tech', 'Entertainment', ...","[Big Tech, Entertainment, GenAI, Google, Langu...","AI-Generated, Big Tech, Entertainment, Gen AI,..."


In [90]:
# merge returned df into original df
merged_df = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
merged_df['title_topic_str'] = merged_df.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)

merged_df


,id,src,title,url,isAI,date,topic_str
0,0,Google News,10 ways to prevent shadow AI disaster,https://www.cio.com/article/2150142/10-ways-to...,True,2024-07-08,10 ways to prevent shadow AI disaster (Topics:...
1,1,Feedly AI,"2 stocks to turn $100 into $1,000 for end of 2024",https://finbold.com/2-stocks-to-turn-100-into-...,True,2024-07-08,"2 stocks to turn $100 into $1,000 for end of 2..."
2,2,Google News,37 ad execs predict 2024 second half trends: A...,https://adage.com/article/agency-news/37-ad-ex...,True,2024-07-08,37 ad execs predict 2024 second half trends: A...
3,3,Google News,3 AI Stocks That Also Pay Dividends,https://investorplace.com/2024/07/3-ai-stocks-...,True,2024-07-08,3 AI Stocks That Also Pay Dividends (Topics: [...
4,4,Google News,3 Stocks to Benefit From the AI Datacenter Pow...,https://www.barrons.com/articles/3-stocks-to-b...,True,2024-07-08,3 Stocks to Benefit From the AI Datacenter Pow...
...,...,...,...,...,...,...,...
136,136,Google News,Will AI Soon Help Animals Talk To Us?,https://mindmatters.ai/2024/07/will-ai-soon-he...,True,2024-07-08,Will AI Soon Help Animals Talk To Us? (Topics:...
137,137,Feedly AI,Worlds first Miss AI crowned and shes a hijab...,https://nypost.com/2024/07/08/lifestyle/worlds...,True,2024-07-08,Worlds first Miss AI crowned and shes a hijab...
138,138,Google News,YouTube Music May Soon Let You Generate Custom...,https://english.jagran.com/technology/youtube-...,True,2024-07-08,YouTube Music May Soon Let You Generate Custom...
139,139,Google News,YouTube Music is piloting 'music any way you l...,https://www.techradar.com/streaming/youtube-mu...,True,2024-07-08,YouTube Music is piloting 'music any way you l...


In [91]:
AIdf = merged_df

### Semantic sort

In [96]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf = AIdf.iloc[sorted_indices] \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

# sort embedding_df to match
embedding_df = embedding_df[sorted_indices]

AIdf


2024-07-08 13:40:05,668 - AInewsbot - INFO - Fetching embeddings for 141 headlines
2024-07-08 13:40:06,313 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


,id,src,title,url,isAI,date,topic_str
0,0,Google News,Heres how Claude 3.5 Sonnet and GPT-4o stack u...,https://www.digitaltrends.com/computing/claude...,True,2024-07-08,Heres how Claude 3.5 Sonnet and GPT-4o stack u...
1,1,Google News,"Why Claude 3.5 Sonnet is the AI to Watch, Not ...",https://www.techopedia.com/news/why-claude-3-5...,True,2024-07-08,"Why Claude 3.5 Sonnet is the AI to Watch, Not ..."
2,2,Google News,Claude AI: A Comprehensive Overview Exploring ...,https://www.marktechpost.com/2024/07/05/claude...,True,2024-07-08,Claude AI: A Comprehensive Overview Exploring ...
3,3,Feedly AI,Exciting Claude 3 Projects with internal knowl...,https://www.geeky-gadgets.com/claude-projects/,True,2024-07-08,Exciting Claude 3 Projects with internal knowl...
4,4,Reddit,University examiners fail to spot ChatGPT answ...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-08,University examiners fail to spot ChatGPT answ...
...,...,...,...,...,...,...,...
136,136,Google News,Can AI Help With Special Ed.? There's Promisea...,https://www.edweek.org/teaching-learning/can-a...,True,2024-07-08,Can AI Help With Special Ed.? There's Promisea...
137,137,Feedly AI,Researchers Warn of Potential for Racial Bias ...,https://www.kqed.org/mindshift/64139/researche...,True,2024-07-08,Researchers Warn of Potential for Racial Bias ...
138,138,Feedly AI,UAB adds programs for students interested inar...,https://www.al.com/educationlab/2024/07/uab-ad...,True,2024-07-08,UAB adds programs for students interested inar...
139,139,Feedly AI,How will the rise of AI in the workplace impac...,https://www.highereddive.com/news/artificial-i...,True,2024-07-08,How will the rise of AI in the workplace impac...


In [98]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(AIdf[["title"]])
    

,title
0,Heres how Claude 3.5 Sonnet and GPT-4o stack up in a direct comparison
1,"Why Claude 3.5 Sonnet is the AI to Watch, Not ChatGPT-4o"
2,Claude AI: A Comprehensive Overview Exploring the Advanced Capabilities and Ethical Design of Anthropics Leading Language Model
3,"Exciting Claude 3 Projects with internal knowledge, 200k context window"
4,University examiners fail to spot ChatGPT answers in real-world test
5,ChatGPT is funnier than humans (and its no joke for professional writers)
6,"How Good Is ChatGPT at Coding, Really? Study finds that while AI can be great, it also struggles due to training limitations"
7,"Go Ahead, Write Your Cover Letter With ChatGPT"
8,Can ChatGPT handle HR? Here's what happened when we put it to the test
9,Can ChatGPT handle HR? Heres what happened when we put it to the test


### Cluster with DBSCAN

In [ ]:
# embedding_model = 'text-embedding-3-large'
# chunksize = 1000
# e_results = []
# for start in range(0, len(ai_history_df), chunksize):
#     tempdf = ai_history_df.iloc[start:start+chunksize]
#     templist = tempdf['title'].tolist()
#     log(f"Fetching embeddings for {len(templist)} headlines starting at row {start}")
#     response = client.embeddings.create(input=templist,
#                                         model=embedding_model)
#     e_results.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))


In [ ]:
# historical_embedding_df = pd.concat(e_results)
# historical_embedding_df.shape


In [ ]:
# historical_embedding_df.to_pickle('historical_embedding_df.pkl')


In [110]:
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])


2024-07-08 13:48:34,239 - AInewsbot - INFO - Fetching embeddings for 141 headlines
2024-07-08 13:48:34,784 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
# reducer = umap.UMAP(n_components=30)  # Reducing to 30 dimensions
# reduced_data = reducer.fit_transform(historical_embedding_df)
# with open('umap_model.pkl', 'wb') as file:
#     pickle.dump(reducer, file)

In [111]:
with open("umap_model.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
    

In [112]:
reduced_data = reducer.transform(embedding_df)


In [116]:
np.isnan(reduced_data).any()


False

In [117]:
# Apply a Clustering Algorithm (e.g., K-Means)
kmeans = KMeans(n_clusters=20)  
clusters = kmeans.fit_predict(reduced_data)

# Evaluate the Clustering
silhouette_avg = silhouette_score(reduced_data, clusters)
print(f'Silhouette Score: {silhouette_avg}')

# Visualization with UMAP (optional)
# reducer_2d = umap.UMAP(n_components=2)  # Reducing to 2 dimensions for visualization
# reduced_data_2d = reducer_2d.fit_transform(embedding_df)

# plt.scatter(reduced_data_2d[:, 0], reduced_data_2d[:, 1], c=clusters, cmap='viridis', s=5)
# plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
# plt.title('UMAP Projection of the News Headlines Clusters')
# plt.show()


Silhouette Score: 0.41552725434303284


In [118]:
dbscan = DBSCAN(eps=0.5, min_samples=3)  # Adjust eps and min_samples as needed
clusters = dbscan.fit_predict(reduced_data)

AIdf['cluster'] = clusters

In [129]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    for i in range(30):
        tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title"]]
        if len(tmpdf) ==0:
            break
        display(tmpdf)
    


,id,title
0,0,Heres how Claude 3.5 Sonnet and GPT-4o stack up in a direct comparison
1,1,"Why Claude 3.5 Sonnet is the AI to Watch, Not ChatGPT-4o"
2,2,Claude AI: A Comprehensive Overview Exploring the Advanced Capabilities and Ethical Design of Anthropics Leading Language Model
3,3,"Exciting Claude 3 Projects with internal knowledge, 200k context window"


,id,title
5,5,ChatGPT is funnier than humans (and its no joke for professional writers)
6,6,"How Good Is ChatGPT at Coding, Really? Study finds that while AI can be great, it also struggles due to training limitations"
7,7,"Go Ahead, Write Your Cover Letter With ChatGPT"
8,8,Can ChatGPT handle HR? Here's what happened when we put it to the test
9,9,Can ChatGPT handle HR? Heres what happened when we put it to the test
10,10,ChatGPT accurately guesses my weight based on a photo. I weigh 167
11,11,I asked chat gpt to depict Bhutans flag in ascii.


,id,title
12,12,"OpenAI, GitHub Beat Digital Copyright Claims From Programmers"
13,13,"Judge Partially Dismisses $1B Lawsuit Against Microsoft, OpenAI, and GitHub Over AI Code Use"
14,14,"Judge dismisses coders' DMCA claims against Microsoft, OpenAI and GitHub"


,id,title
15,15,Micro-agent: make an AI write code until it passes an unit test
17,17,The Evolution of Text-to-Code and Pseudocode Automation
18,18,Transforming Text to Code: An Approach to Efficient Agile Development


,id,title
26,26,Aidan Gomez's Cohere Reaches $5 Billion Valuation with Nvidia's Backing
29,29,Synerise Raises $8.5 Million to Expand AI-Driven Behavioral Modeling Solutions
30,30,Hebbia Raises $130 Million for AI That Helps Firms Answer Complex Questions
31,31,"New York-based Hebbia, which uses AI to help companies sift through documents, raised $130M led by a16z, a source says at a ~$700M post-money valuation (Rachel Metz/Bloomberg)"


...

,id,title
82,82,UK Future Workforce Index 2024: 83% of UK businesses willing to pay a higher wage to those with AI skills amid hiring challenges
83,83,Artificial Intelligence Boom Lifts Paychecks for CIOs
84,84,China sees changing of career guard as AI beats out finance in pay pyramid


,id,title
46,46,China AI leader iFlytek dips into red under 'ultimate' U.S. pressure
96,96,China's AI Leaders Alibaba and SenseTime Showcase Breakthroughs at Shanghai Conference
97,97,China flexes AI muscle at Shanghai expo


,id,title
110,110,MIT researchers introduce generative AI for databases
111,111,MIT researchers introduce generative AI for databases | MIT News | Massachusetts Institute of Technology
113,113,myGenerativeEiAiAlgorithm


,id,title
120,120,"The Download: vacation planning with AI, and smaller models"
121,121,AI Trip Planning: How to Use AI to Plan Your Next Vacation
122,122,How to use AI to plan your next vacation
123,123,Everything You Need to Know About How Artificial Intelligence Transforming Travel and Tourism Industry


,id,title
129,129,South Florida hospital using AI in labor and delivery ward
130,130,"AI can enhance health care and enable cost savings, lawmaker says"
131,131,AI-Driven Behavior Change Could Transform Health Care
132,132,"How AI And New Care Models Make Healthcare Easier, Cheaper, And Better"


# Save and email headlines


In [120]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.src}]({row.url})")
    html_str += f'{row.Index}.<a href="{row.url}">{row.title} - {row.src}</a><br />\n'


2024-07-08 13:51:22,242 - AInewsbot - INFO - [0. Heres how Claude 3.5 Sonnet and GPT-4o stack up in a direct comparison - Google News](https://www.digitaltrends.com/computing/claude-sonnet-vs-gpt-4o-comparison/)
2024-07-08 13:51:22,245 - AInewsbot - INFO - [1. Why Claude 3.5 Sonnet is the AI to Watch, Not ChatGPT-4o - Google News](https://www.techopedia.com/news/why-claude-3-5-sonnet-is-the-ai-to-watch-not-chatgpt-4o)
2024-07-08 13:51:22,246 - AInewsbot - INFO - [2. Claude AI: A Comprehensive Overview Exploring the Advanced Capabilities and Ethical Design of Anthropics Leading Language Model - Google News](https://www.marktechpost.com/2024/07/05/claude-ai-a-comprehensive-overview-exploring-the-advanced-capabilities-and-ethical-design-of-anthropics-leading-language-model/)
2024-07-08 13:51:22,247 - AInewsbot - INFO - [3. Exciting Claude 3 Projects with internal knowledge, 200k context window - Feedly AI](https://www.geeky-gadgets.com/claude-projects/)
2024-07-08 13:51:22,248 - AInewsbot

2024-07-08 13:51:22,260 - AInewsbot - INFO - [35. Teslas upcoming earnings report has to divulge its AI plans and Nvidias role - Feedly AI](https://www.marketwatch.com/story/teslas-upcoming-earnings-report-needs-to-divulge-its-ai-plans-and-nvidias-role-16cde94e)
2024-07-08 13:51:22,260 - AInewsbot - INFO - [36. AI industry needs annual revenues of more than the UAEs GDP to offset costs - Feedly AI](https://readwrite.com/ai-must-find-annual-revenue-more-than-the-gdp-of-singapore-to-justify-expenditure/)
2024-07-08 13:51:22,260 - AInewsbot - INFO - [37. Big Tech needs to generate $600 billion in annual revenue to justify AI hardware expenditure - Reddit](https://www.reddit.com/r/technews/comments/1dxtt72/big_tech_needs_to_generate_600_billion_in_annual/)
2024-07-08 13:51:22,260 - AInewsbot - INFO - [38. Arista Networks: Nvidia Isn't The Only AI Gold Rush Beneficiary (NYSE:ANET) - Google News](https://seekingalpha.com/article/4702847-arista-networks-nvidia-isnt-the-only-ai-gold-rush-benef

2024-07-08 13:51:22,268 - AInewsbot - INFO - [71. Google's AI Aftershock: Expert Explains How to Thrive in the New Search Landscape - HackerNoon](https://hackernoon.com/googles-ai-aftershock-expert-explains-how-to-thrive-in-the-new-search-landscape)
2024-07-08 13:51:22,268 - AInewsbot - INFO - [72. Google Experts Warn That AI May Distort Reality, While AI Overviews Repel Mobile Users - Google News](https://www.cnet.com/tech/computing/google-experts-warn-that-ai-may-distort-reality-while-ai-overviews-repel-mobile-users/)
2024-07-08 13:51:22,268 - AInewsbot - INFO - [73. China's AI model glut is a 'significant waste of resources' due to scarce real-world applications for 100+ LLMs says Baidu CEO - Reddit](https://www.reddit.com/r/technology/comments/1dxwfzs/chinas_ai_model_glut_is_a_significant_waste_of/)
2024-07-08 13:51:22,268 - AInewsbot - INFO - [74. Google's AI search summaries use 10x more energy than just doing a normal Google search - Reddit](https://www.reddit.com/r/technology/c

2024-07-08 13:51:22,273 - AInewsbot - INFO - [103. DRLQ: A Novel Deep Reinforcement Learning (DRL)-based Technique for Task Placement in Quantum Cloud Computing Environments - Feedly AI](https://www.marktechpost.com/2024/07/08/drlq-a-novel-deep-reinforcement-learning-drl-based-technique-for-task-placement-in-quantum-cloud-computing-environments/)
2024-07-08 13:51:22,273 - AInewsbot - INFO - [104. MoBot uses AI to learn how trees move. And to save millions of plant specimens. - Feedly AI](https://www.stltoday.com/news/local/metro/mobot-uses-ai-to-learn-how-trees-move-and-to-save-millions-of-plant-specimens/article_fb9f9660-3885-11ef-8f5a-57b61f2acc14.html)
2024-07-08 13:51:22,273 - AInewsbot - INFO - [105. MoBot uses AI to save millions of plant specimens - Google News](https://www.stltoday.com/news/local/metro/mobot-uses-ai-to-learn-how-trees-move-and-to-save-millions-of-plant-specimens/article_fb9f9660-3885-11ef-8f5a-57b61f2acc14.html)
2024-07-08 13:51:22,273 - AInewsbot - INFO - [106

2024-07-08 13:51:22,281 - AInewsbot - INFO - [137. Researchers Warn of Potential for Racial Bias in AI Apps in the Classroom - Feedly AI](https://www.kqed.org/mindshift/64139/researchers-warn-of-potential-for-racial-bias-in-ai-apps-in-the-classroom)
2024-07-08 13:51:22,281 - AInewsbot - INFO - [138. UAB adds programs for students interested inartificial intelligence - Feedly AI](https://www.al.com/educationlab/2024/07/uab-adds-programs-for-students-interested-in-artificial-intelligence.html)
2024-07-08 13:51:22,281 - AInewsbot - INFO - [139. How will the rise of AI in the workplace impact liberal arts education? - Feedly AI](https://www.highereddive.com/news/artificial-intelligence-liberal-arts-education/720640/)
2024-07-08 13:51:22,281 - AInewsbot - INFO - [140. Mary Meeker urges higher ed to lead in AI - Google News](https://www.insidehighered.com/news/tech-innovation/artificial-intelligence/2024/07/08/mary-meeker-urges-higher-ed-lead-ai)


In [121]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [122]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-08 13:51:30,159 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [124]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.url, row.title))
    

2024-07-08 13:52:10,566 - AInewsbot - INFO - Queuing URLs for scraping


In [125]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-07-08 13:52:11,426 - AInewsbot - INFO - fetching 141 pages using 4 browsers
2024-07-08 13:52:11,429 - AInewsbot - INFO - get_driver - 49319 Initializing webdriver
2024-07-08 13:52:11,431 - AInewsbot - INFO - get_driver - 49319 Initializing webdriver
2024-07-08 13:52:11,431 - AInewsbot - INFO - get_driver - 49319 Initializing webdriver
2024-07-08 13:52:11,432 - AInewsbot - INFO - get_driver - 49319 Initializing webdriver
2024-07-08 13:52:31,947 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-08 13:52:31,947 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-08 13:52:31,947 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-08 13:52:31,947 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-08 13:52:31,948 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-08 13:52:31,948 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-08 13:52:31,948 - AInewsbot - INFO 

2024-07-08 13:54:04,627 - AInewsbot - INFO - get_url(https://www.reddit.com/r/ChatGPT/comments/1dxmwx1/i_asked_chat_gpt_to_depict_bhutans_flag_in_ascii/) - starting get_files https://www.reddit.com/r/ChatGPT/comments/1dxmwx1/i_asked_chat_gpt_to_depict_bhutans_flag_in_ascii/
2024-07-08 13:54:14,103 - AInewsbot - INFO - get_url(Can ChatGPT handle HR? Here's what happened when we put it to the test) - Saving Can_ChatGPT_handle_HR__Here_s_what_happened_when_we_put_it_to_the_test_20240708_135414.html as utf-8
2024-07-08 13:54:14,105 - AInewsbot - INFO - Processing https://news.bloomberglaw.com/litigation/openai-github-beat-digital-copyright-claims-from-programmers
2024-07-08 13:54:14,106 - AInewsbot - INFO - get_url(https://news.bloomberglaw.com/litigation/openai-github-beat-digital-copyright-claims-from-programmers) - starting get_files https://news.bloomberglaw.com/litigation/openai-github-beat-digital-copyright-claims-from-programmers
2024-07-08 13:54:14,304 - AInewsbot - INFO - get_url(

2024-07-08 13:54:49,007 - AInewsbot - INFO - get_url(Exiger Leads Market in AI-Driven Multi-Tier Visibility with Versed AI Acquisition) - Saving Exiger_Leads_Market_in_AI-Driven_Multi-Tier_Visibility_with_Versed_AI_Acquisition_20240708_135449.html as utf-8
2024-07-08 13:54:49,009 - AInewsbot - INFO - Processing https://aithority.com/machine-learning/openai-startup-fund-and-thrive-global-launch-thrive-ai-health-offering-a-hyper-personalized-ai-health-coach/
2024-07-08 13:54:49,010 - AInewsbot - INFO - get_url(https://aithority.com/machine-learning/openai-startup-fund-and-thrive-global-launch-thrive-ai-health-offering-a-hyper-personalized-ai-health-coach/) - starting get_files https://aithority.com/machine-learning/openai-startup-fund-and-thrive-global-launch-thrive-ai-health-offering-a-hyper-personalized-ai-health-coach/
2024-07-08 13:54:52,037 - AInewsbot - INFO - get_url(How AI is driving the venture capital market) - Saving How_AI_is_driving_the_venture_capital_market_20240708_135452

2024-07-08 13:55:27,579 - AInewsbot - INFO - Processing https://readwrite.com/ai-must-find-annual-revenue-more-than-the-gdp-of-singapore-to-justify-expenditure/
2024-07-08 13:55:27,580 - AInewsbot - INFO - get_url(https://readwrite.com/ai-must-find-annual-revenue-more-than-the-gdp-of-singapore-to-justify-expenditure/) - starting get_files https://readwrite.com/ai-must-find-annual-revenue-more-than-the-gdp-of-singapore-to-justify-expenditure/
2024-07-08 13:55:27,801 - AInewsbot - INFO - get_url(Ethereum or Solana? We asked ChatGPT-4o which stock is a better buy for 2024) - Saving Ethereum_or_Solana__We_asked_ChatGPT-4o_which_stock_is_a_better_buy_for_2024_20240708_135527.html as utf-8
2024-07-08 13:55:27,802 - AInewsbot - INFO - Processing https://www.reddit.com/r/technews/comments/1dxtt72/big_tech_needs_to_generate_600_billion_in_annual/
2024-07-08 13:55:27,802 - AInewsbot - INFO - get_url(https://www.reddit.com/r/technews/comments/1dxtt72/big_tech_needs_to_generate_600_billion_in_annu

2024-07-08 13:56:00,824 - AInewsbot - INFO - get_url(https://www.androidpolice.com/galaxy-ai-live-translate-whatsapp-calls/) - starting get_files https://www.androidpolice.com/galaxy-ai-live-translate-whatsapp-calls/
2024-07-08 13:56:03,416 - AInewsbot - INFO - get_url(Samsung flags better-than-expected profit rise as AI boom lifts chip prices) - Saving Samsung_flags_better-than-expected_profit_rise_as_AI_boom_lifts_chip_prices_20240708_135603.html as utf-8
2024-07-08 13:56:03,418 - AInewsbot - INFO - Processing https://www.tomsguide.com/phones/samsung-phones/whatsapp-could-soon-get-a-galaxy-ai-powered-live-translate-feature
2024-07-08 13:56:03,418 - AInewsbot - INFO - get_url(https://www.tomsguide.com/phones/samsung-phones/whatsapp-could-soon-get-a-galaxy-ai-powered-live-translate-feature) - starting get_files https://www.tomsguide.com/phones/samsung-phones/whatsapp-could-soon-get-a-galaxy-ai-powered-live-translate-feature
2024-07-08 13:56:04,607 - AInewsbot - INFO - get_url(Southeast

...

2024-07-08 13:57:52,758 - AInewsbot - INFO - get_url(Artificial Intelligence Boom Lifts Paychecks for CIOs) - Saving Artificial_Intelligence_Boom_Lifts_Paychecks_for_CIOs_20240708_135752.html as utf-8
2024-07-08 13:57:52,761 - AInewsbot - INFO - Processing https://fortune.com/2024/07/08/majority-cfos-admit-dont-fully-understand-ai/
2024-07-08 13:57:52,762 - AInewsbot - INFO - get_url(https://fortune.com/2024/07/08/majority-cfos-admit-dont-fully-understand-ai/) - starting get_files https://fortune.com/2024/07/08/majority-cfos-admit-dont-fully-understand-ai/
2024-07-08 13:57:55,152 - AInewsbot - INFO - get_url(UK Future Workforce Index 2024: 83% of UK businesses willing to pay a higher wage to those with AI skills amid hiring challenges) - Saving UK_Future_Workforce_Index_2024__83__of_UK_businesses_willing_to_pay_a_higher_wage_to_those_with_AI_skills_amid_hiring_challenges_20240708_135755.html as utf-8
2024-07-08 13:57:55,154 - AInewsbot - INFO - Processing https://techcrunch.com/2024/07

2024-07-08 13:58:21,141 - AInewsbot - INFO - get_url(Google claims new AI training tech is 13x faster, 10x more power efficient) - Saving Google_claims_new_AI_training_tech_is_13x_faster__10x_more_power_efficient_20240708_135821.html as utf-8
2024-07-08 13:58:21,142 - AInewsbot - INFO - Processing https://asiatimes.com/2024/07/china-flexes-ai-muscle-at-shanghai-expo/
2024-07-08 13:58:21,142 - AInewsbot - INFO - get_url(https://asiatimes.com/2024/07/china-flexes-ai-muscle-at-shanghai-expo/) - starting get_files https://asiatimes.com/2024/07/china-flexes-ai-muscle-at-shanghai-expo/
2024-07-08 13:58:24,872 - AInewsbot - INFO - get_url(UN Agency: China Leading AI Patents Race) - Saving UN_Agency__China_Leading_AI_Patents_Race_20240708_135824.html as utf-8
2024-07-08 13:58:24,873 - AInewsbot - INFO - Processing https://www.tomshardware.com/pc-components/gpus/chinese-gpu-maker-moore-threads-can-now-scale-to-10000-processors-for-ai-clusters-mtlink-fabric-tech-competes-with-nvidias-nvlink
2024

2024-07-08 13:58:57,898 - AInewsbot - INFO - Processing https://timesofindia.indiatimes.com/world/us/meet-hijab-clad-kenza-layli-from-morcco-worlds-first-ever-miss-ai/articleshow/111581841.cms
2024-07-08 13:58:57,898 - AInewsbot - INFO - get_url(https://timesofindia.indiatimes.com/world/us/meet-hijab-clad-kenza-layli-from-morcco-worlds-first-ever-miss-ai/articleshow/111581841.cms) - starting get_files https://timesofindia.indiatimes.com/world/us/meet-hijab-clad-kenza-layli-from-morcco-worlds-first-ever-miss-ai/articleshow/111581841.cms
2024-07-08 13:58:58,547 - AInewsbot - INFO - get_url(MoBot uses AI to save millions of plant specimens) - Saving MoBot_uses_AI_to_save_millions_of_plant_specimens_20240708_135858.html as utf-8
2024-07-08 13:58:58,550 - AInewsbot - INFO - Processing https://nypost.com/2024/07/08/lifestyle/worlds-first-miss-ai-crowned-kenza-layli-from-morocco-genuinely-excited/
2024-07-08 13:58:58,550 - AInewsbot - INFO - get_url(https://nypost.com/2024/07/08/lifestyle/wor

2024-07-08 13:59:31,045 - AInewsbot - INFO - get_url(https://www.technologyreview.com/2024/07/08/1094742/the-download-vacation-planning-with-ai-and-smaller-models/) - starting get_files https://www.technologyreview.com/2024/07/08/1094742/the-download-vacation-planning-with-ai-and-smaller-models/
2024-07-08 13:59:34,379 - AInewsbot - INFO - get_url(How prompting and integration AI assistants have sped up viral video content production 100 fold) - Saving How_prompting_and_integration_AI_assistants_have_sped_up_viral_video_content_production_100_fold_20240708_135934.html as utf-8
2024-07-08 13:59:34,382 - AInewsbot - INFO - Processing https://www.travelawaits.com/2993617/ai-trip-planning-how-to-use-ai-to-plan-your-next-vacation/
2024-07-08 13:59:34,382 - AInewsbot - INFO - get_url(https://www.travelawaits.com/2993617/ai-trip-planning-how-to-use-ai-to-plan-your-next-vacation/) - starting get_files https://www.travelawaits.com/2993617/ai-trip-planning-how-to-use-ai-to-plan-your-next-vacatio

2024-07-08 14:00:10,972 - AInewsbot - INFO - Processing https://cdt.org/event/u-s-access-board-information-session-foundation-on-artificial-intelligence-and-disability/
2024-07-08 14:00:10,972 - AInewsbot - INFO - get_url(https://cdt.org/event/u-s-access-board-information-session-foundation-on-artificial-intelligence-and-disability/) - starting get_files https://cdt.org/event/u-s-access-board-information-session-foundation-on-artificial-intelligence-and-disability/
2024-07-08 14:00:11,673 - AInewsbot - INFO - get_url(AI can enhance health care and enable cost savings, lawmaker says) - Saving AI_can_enhance_health_care_and_enable_cost_savings__lawmaker_says_20240708_140011.html as utf-8
2024-07-08 14:00:11,673 - AInewsbot - INFO - Processing https://www.govtech.com/artificial-intelligence/government-partners-plan-together-for-accessible-ai
2024-07-08 14:00:11,674 - AInewsbot - INFO - get_url(https://www.govtech.com/artificial-intelligence/government-partners-plan-together-for-accessible

In [126]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'url', 'title', 'path']
pages_df

,id,url,title,path
0,0,https://www.digitaltrends.com/computing/claude...,Heres how Claude 3.5 Sonnet and GPT-4o stack u...,htmldata/Heres_how_Claude_3_5_Sonnet_and_GPT-4...
1,6,https://www.reddit.com/r/technology/comments/1...,"How Good Is ChatGPT at Coding, Really? Study f...",htmldata/How_Good_Is_ChatGPT_at_Coding__Really...
2,8,https://www.fastcompany.com/91147661/can-chatg...,Can ChatGPT handle HR? Here's what happened wh...,htmldata/Can_ChatGPT_handle_HR__Here_s_what_ha...
3,12,https://news.bloomberglaw.com/litigation/opena...,"OpenAI, GitHub Beat Digital Copyright Claims F...",htmldata/OpenAI__GitHub_Beat_Digital_Copyright...
4,16,https://hackernoon.com/converting-epicsstories...,Converting Epics/Stories into Pseudocode using...,htmldata/Converting_Epics_Stories_into_Pseudoc...
...,...,...,...,...
136,123,https://www.travelandtourworld.com/news/articl...,Everything You Need to Know About How Artifici...,htmldata/Everything_You_Need_to_Know_About_How...
137,127,https://www.reddit.com/r/ChatGPT/comments/1dxq...,Ai is fun lol,htmldata/Ai_is_fun_lol_20240708_140001.html
138,131,https://time.com/6994739/ai-behavior-change-he...,AI-Driven Behavior Change Could Transform Heal...,htmldata/AI-Driven_Behavior_Change_Could_Trans...
139,135,https://www.cnn.com/2024/07/08/tech/ai-assisti...,We dont want to leave people behind: AI is hel...,htmldata/We_dont_want_to_leave_people_behind__...


In [127]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [128]:
AIdf

,id,src,title,url,isAI,date,topic_str,cluster,path
0,0,Google News,Heres how Claude 3.5 Sonnet and GPT-4o stack u...,https://www.digitaltrends.com/computing/claude...,True,2024-07-08,Heres how Claude 3.5 Sonnet and GPT-4o stack u...,0,htmldata/Heres_how_Claude_3_5_Sonnet_and_GPT-4...
1,1,Google News,"Why Claude 3.5 Sonnet is the AI to Watch, Not ...",https://www.techopedia.com/news/why-claude-3-5...,True,2024-07-08,"Why Claude 3.5 Sonnet is the AI to Watch, Not ...",0,htmldata/Why_Claude_3_5_Sonnet_is_the_AI_to_Wa...
2,2,Google News,Claude AI: A Comprehensive Overview Exploring ...,https://www.marktechpost.com/2024/07/05/claude...,True,2024-07-08,Claude AI: A Comprehensive Overview Exploring ...,0,htmldata/Claude_AI__A_Comprehensive_Overview_E...
3,3,Feedly AI,Exciting Claude 3 Projects with internal knowl...,https://www.geeky-gadgets.com/claude-projects/,True,2024-07-08,Exciting Claude 3 Projects with internal knowl...,0,htmldata/Exciting_Claude_3_Projects_with_inter...
4,4,Reddit,University examiners fail to spot ChatGPT answ...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-08,University examiners fail to spot ChatGPT answ...,-1,htmldata/University_examiners_fail_to_spot_Cha...
...,...,...,...,...,...,...,...,...,...
136,136,Google News,Can AI Help With Special Ed.? There's Promisea...,https://www.edweek.org/teaching-learning/can-a...,True,2024-07-08,Can AI Help With Special Ed.? There's Promisea...,-1,htmldata/Can_AI_Help_With_Special_Ed___There_s...
137,137,Feedly AI,Researchers Warn of Potential for Racial Bias ...,https://www.kqed.org/mindshift/64139/researche...,True,2024-07-08,Researchers Warn of Potential for Racial Bias ...,-1,htmldata/Researchers_Warn_of_Potential_for_Rac...
138,138,Feedly AI,UAB adds programs for students interested inar...,https://www.al.com/educationlab/2024/07/uab-ad...,True,2024-07-08,UAB adds programs for students interested inar...,-1,htmldata/UAB_adds_programs_for_students_intere...
139,139,Feedly AI,How will the rise of AI in the workplace impac...,https://www.highereddive.com/news/artificial-i...,True,2024-07-08,How will the rise of AI in the workplace impac...,-1,htmldata/How_will_the_rise_of_AI_in_the_workpl...


# Summarize individual pages

In [130]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will output Markdown format.
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [131]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [132]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-07-08 14:02:40,590 - AInewsbot - INFO - Starting summarize
2024-07-08 14:02:49,483 - AInewsbot - INFO - Received 141 summaries


(0, {'id': 'chatcmpl-9imzGOkoHL1jPPZTNBgDt8DFmU33B', 'object': 'chat.completion', 'created': 1720461766, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '- Claude 3.5 Sonnet and GPT-4o are AI assistants with impressive capabilities\n- Claude offers advantages like a larger context window, emphasis on accuracy, and ethical judgments\n- GPT-4o provides a wider range of functions, image generation, flexible log-in options, and more creative content capabilities'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 4227, 'completion_tokens': 65, 'total_tokens': 4292}, 'system_fingerprint': None})


In [133]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

141

In [142]:
markdown_str = ''

for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title} - {row.src}]({row.url})  \n\n {row.topic_str} \n\n{response_dict[row.id]} \n\n"
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. Heres how Claude 3.5 Sonnet and GPT-4o stack up in a direct comparison (Topics: ['Chatgpt', 'Claude 3.5', 'Comparison', 'Gen AI', 'Gpt-4O', 'Language Models', 'Openai', 'Sonnet']) - Google News](https://www.digitaltrends.com/computing/claude-sonnet-vs-gpt-4o-comparison/)  

- Claude 3.5 Sonnet and GPT-4o are AI assistants with impressive capabilities
- Claude offers advantages like a larger context window, emphasis on accuracy, and ethical judgments
- GPT-4o provides a wider range of functions, image generation, flexible log-in options, and more creative content capabilities 



[2. Why Claude 3.5 Sonnet is the AI to Watch, Not ChatGPT-4o (Topics: ['Claude', 'Claude 3.5 Sonnet', 'Gen AI', 'Technology Trends']) - Google News](https://www.techopedia.com/news/why-claude-3-5-sonnet-is-the-ai-to-watch-not-chatgpt-4o)  

- Claude 3.5 Sonnet is gaining popularity in the AI space, surpassing ChatGPT-4o in coding, reasoning, and visual processing.
- Claude 3.5 Sonnet offers features like the Artifacts tool for generating diverse content and plans for memory features and expanded models.
- Anthropic has emphasized security and privacy for Claude 3.5 Sonnet, undergoing testing by external experts and implementing rigorous policy feedback mechanisms. 



[3. Claude AI: A Comprehensive Overview Exploring the Advanced Capabilities and Ethical Design of Anthropics Leading Language Model (Topics: ['Anthropic', 'Claude', 'Claude AI', 'Cohere', 'Ethical Design', 'Ethical Issues', 'Language Models', 'Llms']) - Google News](https://www.marktechpost.com/2024/07/05/claude-ai-a-comprehensive-overview-exploring-the-advanced-capabilities-and-ethical-design-of-anthropics-leading-language-model/)  

- Claude AI is a large language model developed by Anthropic, known for its stringent AI ethics and safe AI systems
- Key features of Claude AI include natural conversations, content creation, language translation, visual processing, and code generation
- Claude AI demonstrates superior performance and safety compared to ChatGPT, with larger context window and ethical guidelines 



[4. Exciting Claude 3 Projects with internal knowledge, 200k context window (Topics: ['Chatgpt', 'Claude', 'Claude 3', 'Code Assistants', 'Cohere', 'Context Window', 'Knowledge', 'Language Models', 'Llms', 'Openai', 'Projects']) - Feedly AI](https://www.geeky-gadgets.com/claude-projects/)  

- Centralized knowledge repository for easy access and reference, custom instructions to guide AI outputs, and 200,000 context window for extensive data integration
- Seamless collaboration and communication through inviting team members to collaborate seamlessly, sharing conversation snapshots, and streamlining communication to keep everyone aligned on project goals
- Versatile use cases for creating style guides, managing code bases, and analyzing interview transcripts, enhanced efficiency and organization with multiple chat threads and ample storage for project knowledge 



[5. University examiners fail to spot ChatGPT answers in real-world test (Topics: ['AI Doom', 'Chatgpt', 'Code Assistants', 'Cohere', 'Education', 'Ethical Issues', 'Examiners', 'Gen AI', 'Language Models', 'Openai', 'Privacy', 'Real-World Test', 'Technology Assessment', 'Testing']) - Reddit](https://www.reddit.com/r/technology/comments/1dxw16s/university_examiners_fail_to_spot_chatgpt_answers/)  

- University examiners failed to detect ChatGPT-generated answers, which accounted for 5% of scripts marked
- Methods like in-person exams without computer access are suggested to prevent AI cheating
- Discussion around the use of AI in exams and the importance of teaching critical thinking and practical skills in education 



...

[137. Can AI Help With Special Ed.? There's Promiseand Reason to Be Cautious (Topics: ['Education', 'Meta', 'Special Education', 'Technology Applications']) - Google News](https://www.edweek.org/teaching-learning/can-ai-help-with-special-ed-theres-promise-and-reason-to-be-cautious/2024/07)  

- Special education practitioners are experimenting with generative artificial intelligence to streamline paperwork and improve instruction for students with disabilities.
- Educators should approach using AI for students with disabilities cautiously due to limited data available and challenges in integrating evidence-based practices into AI tools.
- AI shows promise in personalized learning for students with disabilities, with tools making personalization easier and offering new ways to support student engagement and learning. 



[138. Researchers Warn of Potential for Racial Bias in AI Apps in the Classroom (Topics: ['AI Doom', 'Classroom Apps', 'Education', 'Ethical Issues', 'Gen AI', 'Racial Bias']) - Feedly AI](https://www.kqed.org/mindshift/64139/researchers-warn-of-potential-for-racial-bias-in-ai-apps-in-the-classroom)  

- In a study involving AI grading of essays, researchers found that the AI model penalized Asian American students more than other students, raising concerns about potential racial bias.
- The researchers discovered differences in how AI and humans scored essays based on race and ethnicity, with Asian American students receiving additional penalties in their scores.
- The study highlights the importance of evaluating AI scoring systems and prompts caution in offloading grading work to robots in educational settings. 



[139. UAB adds programs for students interested inartificial intelligence (Topics: ['AI Job Market', 'Education', 'Education Programs', 'Gen AI', 'Meta', 'Students', 'Uab']) - Feedly AI](https://www.al.com/educationlab/2024/07/uab-adds-programs-for-students-interested-in-artificial-intelligence.html)  

- UAB is launching a Master of Science in Artificial Intelligence in Medicine program in spring 2025 as part of efforts to integrate AI in healthcare.
- The program aims to impact healthcare by improving workforce retention, reducing costs, enhancing service quality, and assisting in diagnoses through AI applications.
- The university plans to expand its AI programs to include a graduate certificate for nurses, a doctoral program, and training for healthcare executives and professionals outside the industry. 



[140. How will the rise of AI in the workplace impact liberal arts education? (Topics: ['AI Economic Impacts', 'Education', 'Gen AI', 'Liberal Arts Education', 'Workplace Impact']) - Feedly AI](https://www.highereddive.com/news/artificial-intelligence-liberal-arts-education/720640/)  

- As AI replaces technical jobs, skills like critical thinking and creativity from liberal arts education will become more valuable in the workforce.
- Liberal arts majors are predicted to be in demand as businesses require individuals to handle ethical considerations and challenges that AI technology cannot address.
- Liberal arts students may have an advantage in the job market over STEM-based students due to their ability to provide a humanistic perspective on technology and address ethical concerns related to AI. 



[141. Mary Meeker urges higher ed to lead in AI (Topics: ['AI Economic Impacts', 'AI Job Market', 'Education', 'Gen AI', 'Higher Education', 'Mary Meeker']) - Google News](https://www.insidehighered.com/news/tech-innovation/artificial-intelligence/2024/07/08/mary-meeker-urges-higher-ed-lead-ai)  

- Mary Meeker urges universities to partner with businesses and adopt AI quickly
- Universities need to adapt to the AI era by finding differentiators and attracting students
- Meeker recommends leveraging AI for personalized learning, administrative tasks, and staying current with AI trends 



In [135]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [136]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [137]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-08 14:02:59,659 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [143]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [144]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-07-08 14:06:47,190 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [145]:
list(json.loads(response.choices[0].message.content).values())[0]

['Claude 3.5 Sonnet vs GPT-4o',
 "Claude 3.5 Sonnet's rise",
 'Claude AI ethical design',
 'ChatGPT in education',
 "ChatGPT's humor",
 "ChatGPT's coding capabilities",
 'ChatGPT in HR',
 'OpenAI and GitHub lawsuit',
 'Micro-Agent for coding',
 'AI-driven behavioral modeling',
 "Hebbia's $130M funding",
 'AI in venture capital',
 'AI in finance',
 'AI in healthcare',
 'AI in HR',
 'AI in education',
 'AI in law enforcement',
 'AI in music',
 'AI in travel',
 'AI and sustainability']

# Final Summary

In [ ]:
markdown_str = ''
for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title} - {row.src}]({row.url})  \n\n"
    if 0 < len(catdict[row.id]) < 11 :
        topicstr = ", ".join(catdict[row.id])
        mdstr += f"Topics: {topicstr}\n\n"
    mdstr += f"{response_dict[row.id]} \n\n"
    display(Markdown(mdstr))
    markdown_str += mdstr
    

In [ ]:
FINAL_SUMMARY_PROMPT = f"""You are a summarization assistant. I will provide a list of today's news stories about AI
and summary bullet points in markdown format. You are tasked with identifying and summarizing the key themes,
common facts, and recurring elements. Your goal is to create a concise summary containing about 20 of the most 
frequently mentioned topics and developments.

Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

Topics: AI startup funding, New AI products

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Examples of important stories:

Major investments and funding rounds
Key technological advancements or breakthroughs
Frequently mentioned companies, organizations, or figures
Notable statements by AI leaders
Any other recurring themes or notable patterns

Instructions:

Read the summary bullet points closely.
Use only information provided in them and provide the most common facts without commentary or elaboration.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.
Focus on the most common elements across the bullet points and group similar items together.
Headers must be as short and simple as possible: use "Health Care" and not "AI developments in Health Care" or "AI in Health Care"
Ensure that you provide at least one link from the provided text for each item in the summary.
You must include at least 10 and no more than 25 items in the summary.

Example Output Format:

# Today's AI News

### Security and Privacy:
- ChatGPT Mac app had a security flaw exposing user conversations in plain text. ([Macworld](https://www.macworld.com/article/2386267/chatgpt-mac-sandboxing-security-flaw-apple-intelligence.html))
- Brazil suspended Meta from using Instagram and Facebook posts for AI training over privacy concerns. ([BBC](https://www.bbc.com/news/articles/c7291l3nvwv))

### Health Care:
- AI can predict Alzheimer's disease with 70% accuracy up to seven years in advance. ([Decrypt](https://decrypt.co/238449/ai-alzheimers-detection-70-percent-accurate-study))
- New AI system detects 13 cancers with 98% accuracy, revolutionizing cancer diagnosis. ([India Express](https://news.google.com/articles/CBMiiAFodHRwczovL2luZGlhbmV4cHJl))

Bullet Points to Summarize:

"""

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": FINAL_SUMMARY_PROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)


In [ ]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


In [ ]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


In [ ]:
log("Finished")


In [ ]:
redirect_dict